# Query Expansion

0) Just some imports

In [1]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [2]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    # DONE
    def computeIDFs(self, documents):
        self.idfs = []
        for term in self.terms:
            occurences = sum([1 if term in doc.tokens else 0 for doc in documents])
            self.idfs.append(math.log2(len(documents) / occurences))
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    # DONE
    def computeCorM(self, documents):
        self.corM = []
        
        for i, term in enumerate(self.terms):
            # get number of each term occurences in documents and sum
            row = [ doc.bow[i] for doc in documents ]
            rowSum = sum([ doc.bow[i]**2 for doc in documents])
            row = [i / (rowSum ** (1 / 2)) for i in row]
            self.corM.append(row)
        
        self.corM = np.asarray(self.corM)
        self.corM = self.corM@self.corM.T
        
        for i in range(len(self.corM)):
            self.corM[i, i] *= -1.0

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [3]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [4]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [5]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    ## DONE: implemented version with number of occurences
    def computeBOW_Representation(self):
        self.bow = []
        for term in self.dictionary.terms:
            self.bow.append(self.tokens.count(term))
    
    ### TODO - complete this method; it should compute a TF representation
    # DONE
    def computeTF_Representation(self):
        self.tf = []
        max_number = max(self.bow)
        for number in self.bow:
            self.tf.append(number / max_number)
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    # DONE
    def computeTF_IDF_Representation(self):
        self.tf_idf = []
        for i, idf in enumerate(self.dictionary.idfs):
            self.tf_idf.append(idf * self.tf[i])
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]

1.4) Compute IDFs here

In [6]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['working', 6.459431618637297], ['www', 6.459431618637297], ['york', 6.459431618637297], ['young', 6.459431618637297], ['zdm', 6.459431618637297]]
[['learning', 0.03316686393519938], ['machine', 0.03316686393519938], ['the', 0.6520766965796931], ['and', 1.067114195858537], ['description', 1.2895066171949852]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [7]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow[:30])
print(documents[0].tf_idf[:30])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1148579046593243, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3648579046593243, 0.0, 0.0, 0.9130191741449233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [8]:
### TODO 
# DONE
def getSimilarity(v1, v2):
    numerator, denominatorV1, denominatorV2 = 0, 0, 0
    for i in range(len(v1)):
        numerator += v1[i] * v2[i]
        denominatorV1 += v1[i] * v1[i]
        denominatorV2 += v2[i] * v2[i]
    return numerator / math.sqrt(denominatorV1 * denominatorV2)

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [9]:
#query = "machine learning"
#query = "academic research"
#query = "international conference"
query = "international conference washington"

In [10]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.4742630764397149 | DOC ID = 76
ICML 2003
 The Twentieth International Conference on Machine Learning (ICML-2003).
 August 21-24, 2003 Washington, DC USA. The Twentieth International ... 


RANK = 2 WITH SIMILARITY = 0.4212440556739863 | DOC ID = 86
International Conferences on Machine Learning and Applications
 The 2002 International Conferences on Machine Learning and Applications The
 2003 International Conferences on Machine Learning and Applications. 


RANK = 3 WITH SIMILARITY = 0.2837956967889809 | DOC ID = 19
1998 International Machine Learning Conference
 (larger version of cover). The Fifteenth International Conference on Machine Learning.
 The on-line schedule for ICML-98 contains links to many of the papers. ... 
 Description: The Fifteenth International Conference on Machine Learning. July 24-27, 1998 in Madison, Wisconsin.


RANK = 4 WITH SIMILARITY = 0.20004541104625062 | DOC ID = 41
ICML-2000
 Seventeenth International Conference on Machine L

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [11]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -1.          0.         ...  0.18898224  0.
   0.        ]
 [ 0.          0.         -1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.18898224  0.         ... -1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -1.        ]]


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [12]:
# query = "machine"
# query = "algorithm"
query = "learning"
# query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    # DONE
    print("Suggestions:")

    index = dictionary.terms.index(query)
        
    m = max(dictionary.corM[index])
    mostCorrelated = 0
    for i in range(len(dictionary.corM[index])):
        if dictionary.corM[index, i] == m:
            mostCorrelated = i
    print(dictionary.terms[mostCorrelated])
        
suggestKeywords(query, dictionary)

Suggestions:
machine


# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [13]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    #DONE
    A = [alpha * i for i in qd.tf_idf]
    
    rel_sum = [0 for i in documents[0].tf_idf]
    nrel_sum = [0 for i in documents[0].tf_idf]
    for doc in documents:
        if doc.doc_id in rel_docs:
            rel_sum = [x + y for x, y in zip(rel_sum, doc.tf_idf)]
        if doc.doc_id in nrel_docs:
            nrel_sum = [x + y for x, y in zip(nrel_sum, doc.tf_idf)]
    
    B = [beta * (1 / len(rel_docs)) * i for i in rel_sum]
    
    C = [gamma * (1 / len(nrel_docs)) * i for i in nrel_sum]
    
    qd.tf_idf = [x + y for x, y in zip(A, B)]
    qd.tf_idf = [x - y for x, y in zip(qd.tf_idf, C)]
    
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

RANK = 1 WITH SIMILARITY = 0.683955988307261 | DOC ID = 29
Machine Learning
 6.858/18.428: Machine Learning. ... This course deals with the following topics:
 Formal models of machine learning; Learning concepts from examples; ... 


RANK = 2 WITH SIMILARITY = 0.6197240559840488 | DOC ID = 36
Machine Learning Group
 [ Bristol CS | Index | Research | ML group | Student projects ] Machine
 Learning Research Group. Overview. The Machine Learning Research ... 
 Description: Research on higher-order concept learning, inductive logic programming, multi-agent learning systems,...


RANK = 3 WITH SIMILARITY = 0.18840197888934002 | DOC ID = 13
UTCS Machine Learning Research Group
 Machine learning is the study of adaptive computational systems that
 improve their performance with experience. The UT Machine Learning ... 
 Description: Research on General Inductive Learning, Inductive Logic Programming, Natural Language Learning, Qualitati...


RANK = 4 WITH SIMILARITY = 0.15545715937255858 | DOC

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [14]:
import nltk
#nltk.download()
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [15]:
synsets = wn.synsets('machine')
print(synsets)

[Synset('machine.n.01'), Synset('machine.n.02'), Synset('machine.n.03'), Synset('machine.n.04'), Synset('machine.n.05'), Synset('car.n.01'), Synset('machine.v.01'), Synset('machine.v.02')]


2.3.3) Display all definitions (.definition()) for synsets (machine)

In [16]:
#TODO
for s in synsets:
    print(s.definition())

any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks
an efficient person
an intricate organization that accomplishes its goals efficiently
a device for overcoming resistance at one point by applying force at some other point
a group that controls the activities of a political party
a motor vehicle with four wheels; usually propelled by an internal combustion engine
turn, shape, mold, or otherwise finish by machinery
make by machinery


2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [17]:
#TODO
for s in synsets:
    print(s.hypernyms())

[Synset('device.n.01')]
[Synset('person.n.01')]
[Synset('organization.n.01')]
[Synset('mechanical_device.n.01')]
[Synset('organization.n.01')]
[Synset('motor_vehicle.n.01')]
[Synset('shape.v.02')]
[Synset('produce.v.02')]


See: http://www.nltk.org/howto/wordnet.html
for more examples